In [25]:
import numpy as np
import itertools
from random import randint
from IPython.display import clear_output
from scipy.stats import entropy
from scipy.special import comb
from math import log, factorial, floor, ceil, isinf, isnan

from LPFulleps import CenteredBinomialFulleps as binomialeps #syntax binomialeps[eta, d]
from LPHalfeps import CenteredBinomialHalfeps as binomialeps_rep2
#centered binomial distribution is called with centered_binom_search(eta)

from May_Results import Maydist, Mayeps, ternaryeps #syntax ternaryeps[omega, d]
#Ternary distribution with May's omega values is called with Maydist[k], 0<=i<=5

from Uniformeps import uniformoptieps as uniformeps #syntax uniformeps[eta, d]
#Uniform distribution is called with uniSearch(eta)

from Bliss import Blissdist, blisseps #syntax blisseps[k, d]

q=3329

# distributions are denoted with arrays of the form [w_0,w_1,...,w_eta].
# distributions need to be of length at least 5, filled up w/ zeroes if necessary.

# parameter sets are stored as dictionaries (referred to as eps). For example, to call upon
# level 3 parameter epsilon_20^{(3)}, you call eps[20, 3].
# If epsilon is a Rep-2 parameter set, eps[30, j], eps[31, j], eps[32, j], eps[33, j]
# exist but are constantly set to 0.0 .

# Each set of parameters can be called upon by utilizing their respective syntax.
# The respective syntaxes differ, depending on the distribution.

# To evaluate a certain distribution "dist" with parameter set "eps" for depth "d", call
# for runtimes: evl_T(dist, d, eps)
# for memory size: evl_L(dist, d, eps)
# the returned array gives you the calculated values PER LEVEL, i.e. 
# evl_T(dist, d, eps)[0] returns the calculated runtime on level 0

# For example, to verify our results for B(3), call
# evl_T(etaSearch(3),7,binomialeps[3, 7])

In [3]:
def pbin(eta,i):
    # centered binomial distribution probabilities
    return comb(2*eta,eta+i)/(2**(2*eta))

def quentropy(A,scale, incomplete=True):
    # scaled entropy. This function automatically completes the input set to 1; should this lead to errors due to rounding, use incomplete=False. Is designed to scale with the scaling factor of the binomial coefficient.
    if scale==0:
        return 0
    for i in range(len(A)):
        A[i]=A[i]/scale
    if incomplete:
        A=A+[1-sum(A)]
    return scale*entropy(A,base=2)

In [4]:
def etaSearch(eta):
    # centered binomial search space where each entry appears pbin(i) many times. Is now called centered_binom_search
    w=max(5,eta+1)*[0]
    c=max(5,eta+1)*[0]
    for i in range(0,eta+1):
        w[i]=pbin(eta,i)
    return w

def centered_binom_search(eta):
    return etaSearch(eta)

def uniSearch(eta):
    # creates a uniformly distributed search space
    List=(eta+1)*[1/(2*eta+1)]
    while len(List)<5:
        List+=[0]
    return List

def terSearch(weight):
    # ternary search space where w[1]=weight
    w=[1-2*weight,weight,0,0,0]
    return w

In [5]:
def concrete_search(dist,N):
    # With this function, we can 
    out=[]
    for i in range(len(dist)):
        out+=[dist[i]*N]
    return out

def concrete_search_eps(dist,d,eps,N):
    w=[dist]
    W=[concrete_search(dist,N)]
    for j in range(1,d):
        leps=leveleps(eps,j)
        w+=[cw(w[j-1],leps)]
        W+=[concrete_search(w[j],N)]
    w+=[[0]]
    for i in range(1,len(dist)):
        w[d]+=[w[d-1][i]/2]
    w[d][0]=1-2*sum(w[d])
    W+=[concrete_search(w[d],N)]
    return w,W

In [6]:
# In general, we use dictionaries for parameter sets. In this program, they are referred to as "eps".

def nulleps(d):
    # creates a parameter set of level depth with entry 0 for all epsilon-entries
    out={}
    for i in (10,20,21,22,30,31,32,33):
        for j in range(1,d+1):
            out[i,j]=0
    return out

def leveleps(eps, level):
    #returns all epsilon entries of a specific level
    out={}
    for i in [10,20,21,22,30,31,32,33]:
        out[i]=eps[i,level]
    return out

def copyeps(eps, epsscale=1, changeeps={},changescale=1):
    #copies a parameter set. Can also be used to combine two sets, i.e. when using the gd search for a new library
    out={}
    for key in eps:
        if changeeps=={}:
            out[key]=eps[key]*epsscale
        else:
            out[key]=eps[key]*epsscale+changeeps[key]*changescale
    return out

def formateps(eps,d, levelfirst=False, reverselevelfirst=False):
    # takes any input eps and turns it into a eps of the epsilon-format.
    out={}
    if levelfirst:
        for j in range(1,d):
            for i in [10,20,21,22,30,31,32,33]:
                if (i,j) in eps:
                    out[i,j]=eps[i,j]
                else:
                    out[i,j]=0
    elif reverselevelfirst:
        for k in range(1,d):
            j=d-k
            for i in [10,20,21,22,30,31,32,33]:
                if (i,j) in eps:
                    out[i,j]=eps[i,j]
                else:
                    out[i,j]=0
    else:
        for i in [10,20,21,22,30,31,32,33]:
            for j in range(1,d):
                if (i,j) in eps:
                    out[i,j]=eps[i,j]
                else:
                    out[i,j]=0
    return out

def roundeps(eps,digits=3):
    # takes an input eps and rounds its entries
    out={}
    for key in eps:
        out[key]=round((10**(digits+1)*eps[key]))/(10**(digits+1))
    return out

def ideps(eps):
    # this function requires no explanation
    return eps

def counteps(eps):
    out=0
    for key in eps:
        if eps[key]!=0:
            out+=1
    return out

def lightningeps(eps, scale, gamma=0.001):
    out={}
    for key in eps:
        if eps[key]==0:
            out[key]=[0]
        else:
            out[key]=[]
            sign=np.sign(eps[key])
            for l in range(1,scale+1):
                out[key]+=[sign*l*gamma]
    return out

def normaleps(eps, gamma):
    out={}
    for key in eps:
        out[key]=np.sign(eps[key])*gamma
    return out

#def truthdict(d, tval10=True, tval20=True, tval21=True, tval22=True, tval30=True, tval31=True, tval32=True, tval33=True):
#    out={}
#    for i in [10,20,21,22,30,31,32,33]:
#        val='tval'+str(i)
#        for j in range(d):
#            out[i,j]=locals()[val]
#    return out
#
#def bintruthdict(d, bstr):
#    tval10=str2bool(bstr[0])
#    tval20=str2bool(bstr[1])
#    tval21=str2bool(bstr[2])
#    tval22=str2bool(bstr[3])
#    tval30=str2bool(bstr[4])
#    tval31=str2bool(bstr[5])
#    tval32=str2bool(bstr[6])
#    tval33=str2bool(bstr[7])
#    return truthdict(d, tval10=tval10, tval20=tval20, tval21=tval21, tval22=tval22, tval30=tval30, tval31=tval31, tval32=tval32, tval33=tval33)

def fullbintruthdict(bstr):
    out={}
    d=floor((len(bstr)-1)/8)
    for j in range(d+1):
        out[10,j+1]=bstr[j*8+0]=='1'
        out[20,j+1]=bstr[j*8+1]=='1'
        out[21,j+1]=bstr[j*8+2]=='1'
        out[22,j+1]=bstr[j*8+3]=='1'
        out[30,j+1]=bstr[j*8+4]=='1'
        out[31,j+1]=bstr[j*8+5]=='1'
        out[32,j+1]=bstr[j*8+6]=='1'
        out[33,j+1]=bstr[j*8+7]=='1'
    return out

In [7]:
def searchspace(w):
    # given an input list w, this function returns the entropy of a search space where, for entries from the scale [-eta,eta] (for eta=len(w)+1), an entry i appears relatively w[i] many times
    i=len(w)
    v=[w[0]]
    for i in range(1,i):
        v=v+[w[i]]
        v=v+[w[i]]
    return entropy(v,base=2)

def representations(w,eps):
    # this function calculates, given a previous level search space list w and a parameter set, the amount of representations from an entry of upper search spaces with entries from lower levels that are distributed according to eps
    rep0=quentropy([eps[10],eps[10],eps[20],eps[20],eps[30],eps[30]],w[0])
    rep1=quentropy([eps[21],eps[21],eps[31],eps[31],(w[1]-2*eps[21]-2*eps[31])/2,(w[1]-2*eps[21]-2*eps[31])/2],w[1],incomplete=False)
    rep2=quentropy([eps[22],eps[22],eps[32],eps[32]],w[2])
    rep3=quentropy([eps[33],eps[33],(w[3]-2*eps[33])/2,(w[3]-2*eps[33])/2],w[3],incomplete=False)
    return (2*rep3+2*rep2+2*rep1+rep0)

# the cw functions calculate, according to the previous search space and a parameter set, the search space for the current level
def cw1(pw, ce):
    return (pw[1]+2*pw[2]+pw[3]-2*ce[33]-2*ce[32]-2*ce[31]+2*ce[10]-4*ce[22])/2

def cw2(pw, ce):
    return (pw[3]+2*pw[4]+2*ce[20]+2*ce[21]+2*ce[22]+2*ce[31]-2*ce[33])/2

def cw3(pw, ce):
    return (2*ce[33]+2*ce[32]+2*ce[31]+2*ce[30])/2

def cw(pw, ce):
    c1=cw1(pw, ce)
    c2=cw2(pw, ce)
    c3=cw3(pw, ce)
    c0=1-2*(c1+c2+c3)
    return [c0,c1,c2,c3,0]

In [8]:
#evl_T and evl_L both take an initial distribution as well as a parameter set and calculate each level's respective Time (or space) to create any search list of said level.

def evl(dist, d, eps, retT=True, retL=True):
    T=[0]
    L=[0]
    w=[dist]
    S=[searchspace(w[0])]
    R=[1]
    for j in range(1,d):
        leps=leveleps(eps,j)
        w+=[cw(w[j-1],leps)]
        T+=[0]
        S+=[searchspace(w[j])]
        R+=[representations(w[j-1],leps)]
        L+=[S[j]-R[j]]
        if j==d-1:
            R=R+[0]
            L=L+[S[j]/2]
            T=T+[S[j]/2]
            if isnan(R[j]) or isinf(R[j]):
                T[j]=-R[j]
            elif isnan(R[d]) or isinf(R[d]):
                T[j]=R[d]
            else:
                T[j]=2*L[d]-R[j]
        if j>1:
            if isnan(R[j]) or isinf(R[j]):
                T[j-1]=-R[j]
            elif isnan(R[j-1]) or isinf(R[j-1]):
                # there appears to be a weird error where R[j-1] can be either NaN or inf, I attribute it to rounding.
                # So far, this error has not occured in the neighborhood of optimal parameters, 
                # so we assume that it has no impact on the optimization itself.
                T[j-1]=-R[j-1]
            else:
                T[j-1]=2*L[j]-(R[j-1]-R[j])
        elif isnan(R[1]) or isinf(R[1]):
            T[0]=-R[1]
        else:
            T[0]=max(0,2*L[1]-1)
    for j in range(d):
        T[j]=max(T[j],L[j+1])
    T[d]=0
    if retT and retL:
        return [T,L]
    elif retT:
        return T
    elif retL:
        return L
    return []

def evl_T(dist, d, eps):
    return evl(dist, d, eps, retT=True, retL=False)

def evl_L(dist, d, eps):
    return evl(dist, d, eps, retT=False, retL=True)

In [22]:
# tableprint returns a latex-compilable table of List sizes and run times for a given distribution and a specific parameter set. If no specific distribution is entered, this function assumes the CBD
# Unless one wants to create latex compilable tables, this part of the code can be ignored in its entirety

def tableprint(name, eps, eta=3, d=4, dist=[], treatzero=" ", ifthrees=True, ifcolor=True, iftop=False, ifbottom=False):
    if dist==[]:
        dist=centered_binom_search(eta)
    pdist=dist #leftover from old version
    P="0" #leftover from old version
    T=evl_T(pdist,d,eps)
    L=evl_L(pdist,d,eps)
    max_T= max(T)
    max_L= max(L)
    for i in range(len(T)):
        if T[i]==max_T:
            T[i]="\mathbf{" + "{:0.0f}".format(ceil(T[i]*1000)) + "}"
        else:
            T[i]="{:0.0f}".format(ceil(T[i]*1000))
        if L[i]==max_L:
            L[i]="\mathbf{" + "{:0.0f}".format(ceil(L[i]*1000)) + "}"
        else:
            L[i]="{:0.0f}".format(ceil(L[i]*1000))
    initeps={}
    for key in eps:
        if eps[key]>0:
            initeps[key]="{:0.0f}".format(eps[key]*1000)
        else:
            initeps[key]=treatzero
    line_segment="\\hhline{~|" + ifstring(ifcolor, ">{\\arrayrulecolor{Gray}}->{\\arrayrulecolor{black}}", nots="~") + "|*{" + ifstring(ifthrees, "8", nots="4") + "}{-}|~|" + ifstring(ifcolor, ">{\\arrayrulecolor{Gray}}->{\\arrayrulecolor{black}}", nots="~") + "}"
    if iftop:
        printtop(ifthrees=ifthrees, ifcolor=ifcolor)
    print("        \\multirow{"+ str(d+1) + "}*{$" + name + "$}")
    print("        &0&\\multicolumn{" + ifstring(ifthrees, "8", nots="4") + "}{c|}{-}&$" + T[0] + "$&$" + L[0] + "$" + "\\\\" + line_segment)
    for j in range(1,d-1):
        print("        &" + str(j) + "&$" + initeps[10,j] + "$&$" + initeps[20,j] + "$&$" + initeps[21,j] + "$&$" + initeps[22,j] + ifstring(ifthrees,"$&$" + initeps[30,j] + "$&$" + initeps[31,j] + "$&$" + initeps[32,j] + "$&$" + initeps[33,j]) + "$&$" + T[j] + "$&$" + L[j] + "$\\\\")
    print("        &" + str(d-1) + "&$" + initeps[10,d-1] + "$&$" + initeps[20,d-1] + "$&$" + initeps[21,d-1] + "$&$" + initeps[22,d-1] + ifstring(ifthrees,"$&$" + initeps[30,d-1] + "$&$" + initeps[31,d-1] + "$&$" + initeps[32,d-1] + "$&$" + initeps[33,d-1]) + "$&$" + T[d-1] + "$&$" + L[d-1] + "$\\\\" + line_segment)
    print("        &" + str(d) + "&\\multicolumn{" + ifstring(ifthrees, "8", nots="4") + "}{c|}{-}&$" + T[d] + "$&$" + L[d] + "$\\\\\\hline")
    if ifbottom:
        printbottom()
        
def printtop(ifthrees=True, ifcolor=True):
    print("% IMPORTANT: put \\usepackage{tabularx}, \\usepackage{multirow}, \\usepackage{hhline}, \\usepackage{xcolor, colortbl}, \\definecolor{Gray}{gray}{0.9} in your preamble")
    print("    \\begin{tabular}{c|" + ifstring(ifcolor, ">{\\columncolor{Gray}}") + "c|c" + ifstring(ifcolor, ">{\\columncolor{Gray}}c",nots="|c") + ifstring(ifcolor, "c",nots="|c") + ifstring(ifcolor, ">{\\columncolor{Gray}}c",nots="|c") + ifstring(ifthrees, ifstring(ifcolor, "c",nots="|c") + ifstring(ifcolor, ">{\\columncolor{Gray}}c",nots="|c") + ifstring(ifcolor, "c",nots="|c") + ifstring(ifcolor, ">{\\columncolor{Gray}}c",nots="|c")) + "|c|" + ifstring(ifcolor, ">{\\columncolor{Gray}}") + "c" + "}")
    print("        $\\mathcal D$&$j$&$\\epsilon_{10}^{(j)}$&$\\epsilon_{20}^{(j)}$&$\\epsilon_{21}^{(j)}$&$\\epsilon_{22}^{(j)}" + ifstring(ifthrees,"$&$\\epsilon_{30}^{(j)}$&$\\epsilon_{31}^{(j)}$&$\\epsilon_{32}^{(j)}$&$\\epsilon_{33}^{(j)}") + "$&$\\mathcal T^{(j)}$&$\\mathcal L^{(j)}$" + "\\\\\\hline")

def printbottom():
    print("    \\end{tabular}")
    
def ifstring(b,s,nots=""):
    if b:
        return s
    else:
        return nots

In [10]:
def drawbits(l):
    # draws l random bits
    out=""
    for i in range(l):
        out+=str(randint(0,1))
    return out

def str2bool(v):
    return v=="1"

def hamming(bstr):
    # yields the hamming weight of a binary vector
    out=0
    for i in range(len(bstr)):
        out+=int(bstr[i])
    return out

def create_randset(d, it1=0, it2=100, it3=100, it4=100, it5=0, it6=0, it7=0, it8=0):
    # a randset is used to dictate which parameters are supposed to be optimized,
    # where one 8-bit string denotes all parameters of a given d.
    
    # for example, the string '11111111' would denote that every parameter of a given level would be optimized,
    # while '11110000' dictates that only the Rep-2 parameters are to be optimized.
    
    # since there are l=8*(d-1) many parameters to optimize, we need strings of length l to dictate which
    # parameters should be optimized on all levels.
    
    # By default, we first optimize 2 parameters per level in 100 iterations (i.e. it2=100), then 3 parameters for 100 times
    # and then 4 parameters for 100 times. In other words, the number dictates the hamming weight of every 8-bit string.
    # We found that, after doing this for several rounds, it becomes more and more unlikely that we find better parameters.
    out=[]
    for ham in range(1,9):
        it=locals()['it'+str(ham)]
        for j in range(it):
            randstr=""
            for k in range(d):
                partstr=8*"0"
                while hamming(partstr)!=ham:
                    partstr=drawbits(8)
                randstr+=partstr
            out+=[randstr]
    return out

In [11]:
# leftover from old version
# unlike with the epsilon libraries, it is trivial to calculate the optimal permutation size, should one require to only permute some but not all Permaway-entries into the irrelevant halt of s||e. OptExppermute calculates the Permutation cost given the Permaway-set and lbda

def TrivExppermute(peta):
    return quentropy([2*peta],1)-2*quentropy([peta],1)

def SmartExppermute(peta,c,lbda=0):
    l=1-lbda
    totalper=quentropy([c],l)+quentropy([c],1)
    goodper=quentropy([c-l*peta],l*(1-peta))+quentropy([c],1-peta)
    return totalper-goodper

def OptExppermute(peta,lbda=0):
    l=1-lbda
    c=l/(1+l)
    return SmartExppermute(peta,c,lbda=lbda)

def UpboundExppermute(a,eta,lbda=0):
    pet=peta(eta,Permaway=range(a,eta+1))
    return OptExppermute(pet,lbda=lbda)

def DistExppermute(dist, Permaway, lbda=0):
    l=1-lbda
    eta=len(dist)-1
    c=(eta+1)*[0]
    for i in Permaway:
        c[i]=l*dist[i]
    peta=2*sum(c)-c[0]
    return quentropy([2*peta],1)-2*quentropy([peta],1)

In [12]:
q=3329

def optimizer(dist, #the only required input asks for the distribution where an optimized parameter set should be found
              d=4, #asks for the depth of the algorithm tree
              rough=10, #rough trades off initialization time of the initeps for thoroughness of the starting point
              gamma=0.001, # in the gd_search round, gamma defines the step size
              delta=0.0001, # delta defines the minimal amount of optimization per gd_search round that has to be reached before the algorithm terminates
              setupdelta=0.0001, # setupdelta says that, given an optimal value optT, what is the minimal improvement that should be made to continue down a given path. Trades off runtime for thoroughness
              gd_delta=0, # same as setupdelta for the gd_search round
              gd_scale=1, # can be used instead of gd_delta for the minimal optimization to scale linearly with the optimal value optT
              gd_round=True, # if set to false, no gd_search round takes place. Should generally be left True
              initeps={}, # if initeps={}, the program takes a setup step for a decent initialization of the output eps. If a decent parameter set is known, it can be used as a starting point for the gd_search round instead, omitting the setup phase
              f=ideps, # f describes the function that modifies the parameter set after each gd_search round. It might be appropriate to set f to roundeps
              lightningiter=1000000, #this defines the upper bound of parameter sets checked when entering the lightning gd_search round. If the lightning gd_search round is not wished for, this should be set to 0 instead
              optruthdict={}, # if left empty, every parameter will be optimized. makes runtimes inconceivable when working with depth >3. In general, it is recommended to only optimize up to four parameters per iteration; this can be achieved by setting optruthdict to either bintruthdict or fullepstruthdict with randomized inputs (where each level has hamming weight at most 4)
              printset=[] # printset gives the observer the opportunity to see which inputs are currently under observation. If left empty, this does nothing. Does not impact the algorithm itself
             ):
    global setupset
    global gd_set
    gd_set={}
    if optruthdict=={}:
        optimizedict=truthdict(d)
    else:
        optimizedict=copyeps(optruthdict)
    for i in [10,20,21,22,30,31,32,33]:
        for j in range(1,d):
            if optimizedict[i,j]:
                gd_set[i,j]=[-gamma,0,gamma]
            else:
                gd_set[i,j]=[0]
    # throughout this program, initeps always refers to the epsilon eps that currently is spectated.
    # initeps can be already predefined, otherwise the first step is to find a starting point. This is a hybrid approach with brute force (setup) and greedy discrete gradient descent (gd_search).
    if initeps=={}:
        opteps=nulleps(d-1)
        #initialize Dictionary for epsilon's eps and find the first optimal T-value. Should a library at any point prove to be worse than the current optimal value, it can be discarded immediately. During the course of this program, optT never increases
        initeps=nulleps(d-1)
        optT=max(evl_T(dist, d,opteps))
        if rough>0:
            setupset={}
            for i in [10,20,21,22,30,31,32,33]:
                for j in range(1,d):
                    if optimizedict[i,j]:
                        setupset[i,j]=range(rough+1)
                    else:
                        setupset[i,j]=[0]
            # the setup round finds a starting point for the gd_search round. Should an initeps be given or no setup Round be asked for (i.e. when rough=0), this step is skipped
            [optT,opteps]=setup([dist]+d*[0],(d+1)*[0],[searchspace(dist)]+d*[0], (d+1)*[0], (d+1)*[0], d, 1, optT, nulleps(d-1), nulleps(d-1), rough=rough, setupdelta=setupdelta, printset=printset)
    else:
        opteps=formateps(initeps,d)
        optT=max(evl_T(dist, d,opteps))
    oldchangeeps=nulleps(d-1)
    while gd_round==True:
        # the actual optimization part. Every round, the current working eps opteps gets improved until the difference of the old and new runtime is too small
        curT=optT
        cureps=copyeps(opteps)
        [optT,opteps,optchangeeps]=gd_search([dist]+d*[5*[0]], (d+1)*[0], [searchspace(dist)]+d*[0], (d+1)*[0], (d+1)*[0], d, 1, optT, opteps, opteps, oldchangeeps, nulleps(d-1), nulleps(d-1), nulleps(d-1),gamma=gamma, gd_delta=gd_delta, gd_scale=gd_scale, lightningiter=lightningiter, printset=printset)
        oldchangeeps=copyeps(optchangeeps)
        opteps=f(opteps)
        optT=max(evl_T(dist,d,opteps))
        if curT-optT<delta:
            gd_round=False
    return opteps

#The Subroutine for finding a good starting point
def setup(w, R, S, T, L, d, curlevel, optT, opteps, initeps, rough=10, setupdelta=0.0001, printset=[]):
    # setup goes through all possible epsilons (in steps of size 1/rough) and finds the optimal starting point for the gd_search round.
    global setupset
    j=curlevel #leftover from old version
    prevlevel=j-1
    nextlevel=j+1
    for a10 in setupset[10,j]:
        b10=a10/rough
        if w[prevlevel][0]/2<b10:
            break
        for a20 in setupset[20,j]:
            b20=a20/rough
            if (w[prevlevel][0]-2*b10)/2<b20:
                break
            for a21 in setupset[21,j]:
                b21=a21/rough
                if w[prevlevel][1]/2<b21:
                    break
                for a22 in setupset[22,j]:
                    b22=a22/rough
                    if w[prevlevel][2]/2<b22:
                        break
                    for a30 in setupset[30,j]:
                        b30=a30/rough
                        if (w[prevlevel][0]-2*b10-2*b20)/2<b30:
                            break
                        for a31 in setupset[31,j]:
                            b31=a31/rough
                            if (w[prevlevel][2]-2*b21)/2<2*b31:
                                break
                            for a32 in setupset[32,j]:
                                b32=a32/rough
                                if (w[prevlevel][2]-2*b22)/2<b32:
                                    break
                                for a33 in setupset[33,j]:
                                    b33=a33/rough
                                    if w[prevlevel][3]<2*b33:
                                        break
                                    initeps[10,j]=b10
                                    initeps[20,j]=b20
                                    initeps[21,j]=b21
                                    initeps[22,j]=b22
                                    initeps[30,j]=b30
                                    initeps[31,j]=b31
                                    initeps[32,j]=b32
                                    initeps[33,j]=b33
                                    leps=leveleps(initeps,j)
                                    w[j]=cw(w[prevlevel],leps)
                                    S[j]=searchspace(w[j])
                                    R[j]=representations(w[prevlevel],leps)
                                    L[j]=S[j]-R[j]
                                    if j>1:
                                        T[prevlevel]=2*L[j]-(R[j-1]-R[j])
                                    else:
                                        T[prevlevel]=max(0,2*(S[1]-R[1])-(1-R[1]*np.log(2)/np.log(q)))
                                    for k in range(j,d+1):
                                        T[k]=0
                                    newT=max(T)
                                    if j==d-1:
                                        R[nextlevel]=0
                                        T[nextlevel]=S[j]/2
                                        L[nextlevel]=S[j]/2
                                        T[j]=2*L[nextlevel]-(R[j]-R[nextlevel])
                                        newT=max(T)
                                        if newT<optT:
                                            clear_output(wait=True)
                                            for entry in printset:
                                                print(entry)
                                            print(initeps)
                                            print("T=" + str(T))
                                            print("best runtime is " + str(newT))
                                            optT=newT
                                            opteps=copyeps(initeps)
                                    elif newT<optT-setupdelta:
                                        [optT,opteps]=setup(w, R, S, T, L, d, nextlevel, optT,opteps,initeps, rough=rough, printset=printset)
    return [optT,opteps]

def gd_search(w, R, S, T, L, d,curlevel,optT, opteps, oldeps, oldchangeeps, initeps, changeeps, optchangeeps, gamma=0.001, gd_delta=0, gd_scale=1, lightningiter=100000, printset=[]):
    # one gd_search round defines a greedy and discrete gradient descent optimization step.
    # given the optimal parameter set and the subset of parameters we want to optimize, find the best neighbouring
    # parameter set where all parameters to not be optimized are equal to their current parameter counterpart
    # and all other parameters differ by at most gamma.
    global gd_set
    j=curlevel #leftover from old version
    prevlevel=j-1
    nextlevel=j+1
    for change10 in gd_set[10,j]: 
        if 0>oldeps[10,j]+change10 or w[prevlevel][0]/2<oldeps[10,j]+change10 or change10*oldchangeeps[10,j]<0:
            # check if new parameter is >=0 AND
            # new parameter does not create more values 1 that exist in previous level AND
            # For faster optimization: check if new parameter does not revert changes from previous parameter actualization
            continue
        for change20 in gd_set[20,j]:
            if 0>oldeps[20,j]+change20 or (w[prevlevel][0]-2*(oldeps[10,j]+change10))/2<oldeps[20,j]+change20 or change20*oldchangeeps[20,j]<0:
                # same as with change10
                continue
            for change21 in gd_set[21,j]:
                if 0>oldeps[21,j]+change21 or w[prevlevel][1]/2<oldeps[21,j]+change21 or change21*oldchangeeps[21,j]<0:
                    # same as with change10
                    continue
                for change22 in gd_set[22,j]:
                    if 0>oldeps[22,j]+change22 or w[prevlevel][2]/2<oldeps[22,j]+change22 or change22*oldchangeeps[22,j]<0:
                        # same as with change10
                        continue
                    for change32 in gd_set[32,j]:
                        if 0>oldeps[32,j]+change32 or (w[prevlevel][2]/2-2*(oldeps[22,j]+change22))/2<oldeps[32,j]+change32 or change32*oldchangeeps[32,j]<0:
                            # same as with change10
                            continue
                        for change33 in gd_set[33,j]:
                            if 0>oldeps[33,j]+change33 or w[prevlevel][3]/2<oldeps[33,j]+change33 or change33*oldchangeeps[33,j]<0:
                                # same as with change10
                                continue
                            for change31 in gd_set[31,j]:
                                if 0>oldeps[31,j]+change31 or (w[prevlevel][1]-2*(oldeps[21,j]+change21))/2<oldeps[31,j]+change31 or change31*oldchangeeps[31,j]<0:
                                    # same as with change10
                                    continue
                                for change30 in gd_set[30,j]:
                                    if 0>oldeps[30,j]+change30 or (w[prevlevel][0]-2*(oldeps[20,j]+change20+oldeps[10,j]+change10))/2<oldeps[30,j]+change30 or change30*oldchangeeps[30,j]<0:
                                        # same as with change10
                                        continue
                                    initeps[10,j]=oldeps[10,j]+change10
                                    initeps[20,j]=oldeps[20,j]+change20
                                    initeps[21,j]=oldeps[21,j]+change21
                                    initeps[22,j]=oldeps[22,j]+change22
                                    initeps[30,j]=oldeps[30,j]+change30
                                    initeps[31,j]=oldeps[31,j]+change31
                                    initeps[32,j]=oldeps[32,j]+change32
                                    initeps[33,j]=oldeps[33,j]+change33
                                    changeeps[10,j]=change10
                                    changeeps[20,j]=change20
                                    changeeps[21,j]=change21
                                    changeeps[22,j]=change22
                                    changeeps[30,j]=change30
                                    changeeps[31,j]=change31
                                    changeeps[32,j]=change32
                                    changeeps[33,j]=change33
                                    leps=leveleps(initeps,j)
                                    w[j]=cw(w[j-1],leps)
                                    S[j]=searchspace(w[j])
                                    R[j]=representations(w[prevlevel],leps)
                                    L[j]=S[j]-R[j]
                                    if j>1:
                                        T[prevlevel]=2*L[j]-(R[j-1]-R[j])
                                    else:
                                        T[prevlevel]=max(0,2*(S[1]-R[1])-(1-R[1]*np.log(2)/np.log(q)))
                                    T[j]=0
                                    for k in range(j+1,d+1):
                                        T[k]=0
                                        R[k]=0
                                        L[k]=0
                                        S[k]=0
                                        w[k]=5*[0]
                                    newT=max(T)
                                    if j==d-1:
                                        R[nextlevel]=0
                                        T[nextlevel]=S[j]/2
                                        L[nextlevel]=S[j]/2
                                        T[j]=2*L[nextlevel]-(R[j]-R[nextlevel])
                                        newT=max(T)
                                        if newT<optT:
                                            clear_output(wait=True)
                                            for entry in printset:
                                                print(entry)
                                            print(initeps)
                                            print("T=" + str(T))
                                            print("best runtime is " + str(newT))
                                            optT=newT
                                            opteps=copyeps(initeps)
                                            optchangeeps=copyeps(changeeps)
                                            notnull=counteps(changeeps)
                                            if notnull>0:
                                                scale=floor(lightningiter**(1/notnull))
                                                if scale>1:
                                                    lightningsets=lightningeps(changeeps,scale,gamma=gamma)
                                                    [optT,opteps,optchangeeps]=lightning_gd([w[0]]+d*[5*[0]], (d+1)*[0], [searchspace(w[0])]+d*[0], (d+1)*[0], (d+1)*[0], d,1, optT, opteps, oldeps, nulleps(d-1), nulleps(d-1), optchangeeps, lightningsets, gamma=gamma, printset=printset)
                                                else:
                                                    clear_output(wait=True)
                                                    for entry in printset:
                                                        print(entry)
                                                    print(initeps)
                                                    print("T=" + str(T))
                                                    print("best runtime is " + str(newT))
                                                    optT=newT
                                                    opteps=copyeps(initeps)
                                                    optchangeeps=copyeps(changeeps)
                                    elif newT<gd_scale*optT-gd_delta:
                                        [optT,opteps,optchangeeps]=gd_search(w, R, S, T, L, d,nextlevel,optT, opteps, oldeps, oldchangeeps, initeps, changeeps, optchangeeps,gamma=gamma, gd_delta=gd_delta, gd_scale=gd_scale, lightningiter=lightningiter, printset=printset)
    return [optT,opteps,optchangeeps]

def lightning_gd(w, R, S, T, L, d,curlevel,optT, opteps, oldeps, initeps, changeeps, optchangeeps, lightningsets, gamma=0.001, printset=[]):
    j=curlevel
    prevlevel=j-1
    nextlevel=j+1
    for change10 in lightningsets[10,j]:
        if 0>oldeps[10,j]+change10 or w[prevlevel][0]/2<oldeps[10,j]+change10:
            continue
        for change20 in lightningsets[20,j]:
            if 0>oldeps[20,j]+change20 or (w[prevlevel][0]-2*(oldeps[10,j]+change10))/2<oldeps[20,j]+change20:
                continue
            for change21 in lightningsets[21,j]:
                if 0>oldeps[21,j]+change21 or w[prevlevel][1]/2<oldeps[21,j]+change21:
                    continue
                for change22 in lightningsets[22,j]:
                    if 0>oldeps[22,j]+change22 or w[prevlevel][2]/2<oldeps[22,j]+change22:
                        continue
                    for change32 in lightningsets[32,j]:
                        if 0>oldeps[32,j]+change32 or (w[prevlevel][2]/2-2*(oldeps[22,j]+change22))/2<oldeps[32,j]+change32:
                            continue
                        for change33 in lightningsets[33,j]:
                            if 0>oldeps[33,j]+change33 or w[prevlevel][3]/2<oldeps[33,j]+change33:
                                continue
                            for change31 in lightningsets[31,j]:
                                if 0>oldeps[31,j]+change31 or (w[prevlevel][1]-2*(oldeps[21,j]+change21))/2<oldeps[31,j]+change31:
                                    continue
                                for change30 in lightningsets[30,j]:
                                    if 0>oldeps[30,j]+change30 or (w[prevlevel][0]-2*(oldeps[20,j]+change20+oldeps[10,j]+change10))/2<oldeps[30,j]+change30:
                                        continue
                                    initeps[10,j]=oldeps[10,j]+change10
                                    initeps[20,j]=oldeps[20,j]+change20
                                    initeps[21,j]=oldeps[21,j]+change21
                                    initeps[22,j]=oldeps[22,j]+change22
                                    initeps[30,j]=oldeps[30,j]+change30
                                    initeps[31,j]=oldeps[31,j]+change31
                                    initeps[32,j]=oldeps[32,j]+change32
                                    initeps[33,j]=oldeps[33,j]+change33
                                    changeeps[10,j]=change10
                                    changeeps[20,j]=change20
                                    changeeps[21,j]=change21
                                    changeeps[22,j]=change22
                                    changeeps[30,j]=change30
                                    changeeps[31,j]=change31
                                    changeeps[32,j]=change32
                                    changeeps[33,j]=change33
                                    leps=leveleps(initeps,j)
                                    w[j]=cw(w[j-1],leps)
                                    S[j]=searchspace(w[j])
                                    R[j]=representations(w[prevlevel],leps)
                                    L[j]=S[j]-R[j]
                                    if j>1:
                                        T[prevlevel]=2*L[j]-(R[j-1]-R[j])
                                    else:
                                        T[prevlevel]=max(0,2*(S[1]-R[1])-(1-R[1]*np.log(2)/np.log(q)))
                                    for k in range(j,d+1):
                                        T[k]=0
                                    newT=max(T)
                                    if j==d-1:
                                        R[nextlevel]=0
                                        T[nextlevel]=S[j]/2
                                        L[nextlevel]=S[j]/2
                                        T[j]=2*L[nextlevel]-(R[j]-R[nextlevel])
                                        newT=max(T)
                                        if newT<optT:
                                            clear_output(wait=True)
                                            for entry in printset:
                                                print(entry)
                                            print(initeps)
                                            print("T=" + str(T))
                                            print("best runtime is " + str(newT))
                                            optT=newT
                                            opteps=copyeps(initeps)
                                            optchangeeps=normaleps(changeeps, gamma)
                                    elif newT<optT:
                                        [optT,opteps,optchangeeps]=lightning_gd(w, R, S, T, L, d, nextlevel, optT, opteps, oldeps, initeps, changeeps, optchangeeps, lightningsets, gamma=gamma, printset=printset)
    return [optT,opteps,optchangeeps]

In [ ]:
# example for an optimizer call. Finds optimal depth 4 Rep-3 parameters for distribution [0.5,0.25,0,0,0]

dist=Maydist[3]

optimizer(dist,
              d=4,
              rough=10,
              gamma=0.001,
              delta=0.0001,
              setupdelta=0.001,
              gd_delta=0, 
              gd_scale=1, 
              gd_round=True, 
              initeps={},
              f=ideps,
              lightningiter=1000000,
              optruthdict=fullbintruthdict(8*'11111111'),
              printset=[]
             )

In [ ]:
# this is the optimization part for CBD
cbdoptieps={}
# randset=[8*'00000000'] #for Rep-0
# randset=[8*'10000000'] #for Rep-1
# randset=[8*'11110000'] #for Rep-2
randset=[8*'11110000']+create_randset(8) #for Rep-3
for eta in range(2,4):
    dist=centered_binom_search(eta)
    cbdoptieps[eta,1]={}
    for d in range(2,9):
        cbdoptieps[eta,d]=cbdoptieps[eta,d-1]
        for randbits in randset:
            initeps=cbdoptieps[eta,d]
            cbdoptieps[eta,d]=optimizer(dist,rough=0,gamma=0.001,optruthdict=fullbintruthdict(randbits),gd_delta=0, initeps=initeps, d=d,f=ideps, printset=["eta=" + str(eta), "d=" + str(d), randbits])

In [ ]:
# this is the optimization part for uniform distributions
uoptieps={}
# randset=[8*'00000000'] #for Rep-0
# randset=[8*'10000000'] #for Rep-1
# randset=[8*'11110000'] #for Rep-2
randset=[8*'11110000']+create_randset(8) #for Rep-3
for eta in range(1,4):
    dist=uniSearch(eta)
    uoptieps[eta,1]={}
    for d in range(2,9):
        uoptieps[eta,d]=uoptieps[eta,d-1]
        for randbits in randset:
            initeps=uoptieps[eta,d]
            uoptieps[eta,d]=optimizer(dist,rough=0,gamma=0.001,optruthdict=fullbintruthdict(randbits),gd_delta=0, initeps=initeps, d=d,f=ideps, printset=["eta=" + str(eta), "d=" + str(d), randbits])

In [ ]:
# this is the optimization part of the code for ternary distributions. 
teroptieps={}
# randset=[8*'00000000'] #for Rep-0
# randset=[8*'10000000'] #for Rep-1
# randset=[8*'11110000'] #for Rep-2
randset=[8*'11110000']+create_randset(8) #for Rep-3
for k in range(6):
    dist=Maydist[k]
    teroptieps[Maydist[k][1],1]={}
    for d in range(2,3):
        teroptieps[Maydist[k][1],d]=teroptieps[Maydist[k][1],d-1]
        for randbits in randset:
            initeps=teroptieps[Maydist[k][1],d]
            teroptieps[Maydist[k][1],d]=optimizer(dist,rough=0,gamma=0.001,optruthdict=fullbintruthdict(randbits),gd_delta=0, initeps=initeps, d=d,f=roundeps, printset=["omega=" + str(2*Maydist[k][1]), "d=" + str(d), randbits])

In [ ]:
# this is the optimization part for BLISS
blissoptieps={}
# randset=[8*'00000000'] #for Rep-0
# randset=[8*'10000000'] #for Rep-1
# randset=[8*'11110000'] #for Rep-2
randset=[8*'11110000']+create_randset(8) #for Rep-3
for eta in (0,1,3,4):
    dist=Blissdist[eta]
    blisseps[eta,1]={}
    for d in range(2,9):
        blisseps[eta,d]=blisseps[eta,d-1]
        for randbits in randset:
            initeps=blisseps[eta,d]
            blisseps[eta,d]=optimizer(dist,rough=0,gamma=0.001,optruthdict=fullbintruthdict(randbits),gd_delta=0, initeps=initeps, d=d,f=roundeps, printset=["eta=" + str(eta), "d=" + str(d), randbits])